# Решающие деревья и kaggle
В этом ноутбуке мы попробуем применить на практике решающие деревья для задачи бинарной классификации. Задание мы возьмём с kaggle. Заодно сделаем свой первый submission на этой платформе.

## Загрузка и анализ данных

Загрузите данные из [этого соревнования](https://www.kaggle.com/c/titanic/overview) kaggle в pandas.DataFrame. Посмотрите, что из себя представляют признаки.

In [8]:
import pandas as pd

data = pd.read_csv('titanic/train.csv')
data.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
463,464,0,2,"Milling, Mr. Jacob Christian",male,48.0,0,0,234360,13.0000,NaN,S
426,427,1,2,"Clarke, Mrs. Charles V (Ada Maria Winfield)",female,28.0,1,0,2003,26.0000,NaN,S
399,400,1,2,"Trout, Mrs. William H (Jessie L)",female,28.0,0,0,240929,12.6500,NaN,S
168,169,0,1,"Baumann, Mr. John D",male,NaN,0,0,PC 17318,25.9250,NaN,S
615,616,1,2,"Herman, Miss. Alice",female,24.0,1,2,220845,65.0000,NaN,S
386,387,0,3,"Goodwin, Master. Sidney Leonard",male,1.0,5,2,CA 2144,46.9000,NaN,S
806,807,0,1,"Andrews, Mr. Thomas Jr",male,39.0,0,0,112050,0.0000,A36,S
125,126,1,3,"Nicola-Yarred, Master. Elias",male,12.0,1,0,2651,11.2417,NaN,C
690,691,1,1,"Dick, Mr. Albert Adrian",male,31.0,1,0,17474,57.0000,B20,S
225,226,0,3,"Berglund, Mr. Karl Ivar Sven",male,22.0,0,0,PP 4348,9.3500,NaN,S


In [7]:
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Есть 3 признака с пропусками. Пропуски каждого из них будем обрабатывать отдельным образом:
1. *Cabin*. Из 800 записей этот признако отсутствует почти у 700. Поэтому выбросим данный признак из рассмотрения.
2. *Embarked*. Этот признак отсутствует у 2 записей. И он категориальный. Будем заполнять пропуски этого признака самым популярным значением.
3. *Age*. Вещественнозначный признак, отсутствующий примерно у одной пятой части записей. В качестве бейзлайна заполним его медианным значением.

## Предобработка данных

1. Заполним пропуски в признаках
2. Уберём текстовые признаки 
3. Преобразуем данные в к numpy-массивам
4. Вынесем значения откликов в отдельный вектор

In [49]:
preproc_data = data.drop(columns="Cabin")

preproc_data["Age"].fillna(preproc_data["Age"].median(), inplace=True)

preproc_data["Embarked"].fillna(preproc_data["Embarked"].value_counts().index[0], inplace=True)

preproc_data.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

Преобразуем признак *Sex*. Закодируем числовым значением каждое из двух возможных текстовых значений этого признака. То же самое сделаем для признака *Embarked*

In [50]:
preproc_data["Sex"] = pd.Categorical(preproc_data["Sex"])
preproc_data["Sex"] = preproc_data["Sex"].cat.codes

preproc_data["Embarked"] = pd.Categorical(preproc_data["Embarked"])
preproc_data["Embarked"] = preproc_data["Embarked"].cat.codes

preproc_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,2
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,2
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.0,0,0,211536,13.0000,2
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.0,0,0,112053,30.0000,2
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,28.0,1,2,W./C. 6607,23.4500,2
889,890,1,1,"Behr, Mr. Karl Howell",1,26.0,0,0,111369,30.0000,0


In [51]:
x_train = preproc_data.drop(columns=["Survived", "Ticket", "Name"]).to_numpy()
y_train = preproc_data["Survived"].to_numpy()
print(x_train.shape, y_train.shape)

(891, 8) (891,)


## Перебор параметров дерева решений.

Обучим классификатор на основе дерева решений. При этом:
1. Переберём различные параметры дерева
2. Используем кросс-валидацию для оценки качества модели
3. Не надо изобретать велосипед. Используйте ф-ю GridSearchCV

In [60]:
%%time

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [i+1 for i in range(16)],
    'min_samples_leaf': [i+1 for i in range(20)],
    'random_state': [1337],
    'min_impurity_decrease': [.0, .1, .3, .5, .7, 1.0, 3.0],
    'ccp_alpha': [0, 0.005, 0.01, 0.015, 0.02, 0.03]
}

clf = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5, n_jobs=-1, verbose=1)
clf.fit(x_train, y_train)

Fitting 5 folds for each of 26880 candidates, totalling 134400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 312 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 18424 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 50424 tasks      | elapsed:   45.1s
[Parallel(n_jobs=-1)]: Done 95224 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 134400 out of 134400 | elapsed:  2.1min finished


CPU times: user 1min 21s, sys: 515 ms, total: 1min 22s
Wall time: 2min 3s


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'ccp_alpha': [0, 0.005, 0.01, 0.015, 0.02, 0.03],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16],
                         'min_impurity_decrease': [0.0, 0.1, 0.3, 0.5, 0.7, 1.0,
                                                   3.0],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                              12, 13, 14, 15, 16, 17, 18, 19,
                                              20],
                         'random_state': [1337]},
             verbose=1)

In [61]:
print("Точность кросс-валидации на лучших параметрах равна %.5f" % clf.best_score_)
print("Лучшие параметры:")
print(clf.best_params_)

Точность кросс-валидации на лучших параметрах равна 0.82380
Лучшие параметры:
{'ccp_alpha': 0, 'criterion': 'entropy', 'max_depth': 6, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 8, 'random_state': 1337}


## Применение обученной на тестовых данных

В этом разделе мы: 
1. Загрузим тестовую выборку
2. Применим к данным ту же предобработку данных, что и для обучающей выборки
3. Сделаем на ней предсказания
4. Завернём результаты предсказания в необходимый для kaggle формат
5. Сохраним эти данные в формате .csv

In [63]:
test_data = pd.read_csv('titanic/test.csv')
test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [64]:
test_preproc_data = test_data.drop(columns="Cabin")

test_preproc_data["Age"].fillna(test_preproc_data["Age"].median(), inplace=True)

test_preproc_data["Embarked"].fillna(test_preproc_data["Embarked"].value_counts().index[0], inplace=True)

test_preproc_data.isna().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           1
Embarked       0
dtype: int64

Оказалось, что в тестовых данных есть пропуск в признаке *Fare*. В обучающей выборке в этом признаке пропуска не было. Мы заполним его медианным значением по всему признаку.

In [65]:
test_preproc_data["Fare"].fillna(test_preproc_data["Fare"].median(), inplace=True)
test_preproc_data.isna().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [66]:
test_preproc_data["Sex"] = pd.Categorical(test_preproc_data["Sex"])
test_preproc_data["Sex"] = test_preproc_data["Sex"].cat.codes

test_preproc_data["Embarked"] = pd.Categorical(test_preproc_data["Embarked"])
test_preproc_data["Embarked"] = test_preproc_data["Embarked"].cat.codes

test_preproc_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,2
2,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,1
3,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,2
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,2
...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",1,27.0,0,0,A.5. 3236,8.0500,2
414,1306,1,"Oliva y Ocana, Dona. Fermina",0,39.0,0,0,PC 17758,108.9000,0
415,1307,3,"Saether, Mr. Simon Sivertsen",1,38.5,0,0,SOTON/O.Q. 3101262,7.2500,2
416,1308,3,"Ware, Mr. Frederick",1,27.0,0,0,359309,8.0500,2


In [68]:
x_test = test_preproc_data.drop(columns=["Ticket", "Name"]).to_numpy()

In [70]:
y_test_pred = clf.predict(x_test)

In [74]:
result_df = pd.DataFrame({
    "PassengerId": test_preproc_data["PassengerId"],
    "Survived": y_test_pred
})

In [77]:
result_df.to_csv('result.csv', index=False)

После загрузки на kaggle это решение получило точность 0.76794

## Счастливые билетики: правда или миф?

Проверьте, зависит ли выживаемость пассажиров от того, являлся ли номер их билета счастливым (сумма первой половины цифр равна сумме второй половины цифр).

In [101]:
def is_lucky_ticket(ticket):
    if ticket == "LINE":
        return False
    print(ticket)
    ticket_number = ticket.split(" ")[-1]
    if sum([int(e) for e in ticket_number[:len(ticket_number)//2]]) ==\
        sum([int(e) for e in ticket_number[len(ticket_number)//2+len(ticket_number)%2:]]):
        return True
    return False

print(is_lucky_ticket("CA 43614"))
print(is_lucky_ticket("CA 41614"))
print(is_lucky_ticket("5135235"))
print(is_lucky_ticket("343"))
print(is_lucky_ticket("3544"))


CA 43614
False
CA 41614
True
5135235
False
343
True
3544
True


In [102]:
preproc_data["LuckyTicket"] = preproc_data["Ticket"].transform(is_lucky_ticket)

A/5 21171
PC 17599
STON/O2. 3101282
113803
373450
330877
17463
349909
347742
237736
PP 9549
113783
A/5. 2151
347082
350406
248706
382652
244373
345763
2649
239865
248698
330923
113788
349909
347077
2631
19950
330959
349216
PC 17601
PC 17569
335677
C.A. 24579
PC 17604
113789
2677
A./5. 2152
345764
2651
7546
11668
349253
SC/Paris 2123
330958
S.C./A.4. 23567
370371
14311
2662
349237
3101295
A/4. 39886
PC 17572
2926
113509
19947
C.A. 31026
2697
C.A. 34651
CA 2144
2669
113572
36973
347088
PC 17605
2661
C.A. 29395
S.P. 3464
3101281
315151
C.A. 33111
CA 2144
S.O.C. 14879
2680
1601
348123
349208
374746
248738
364516
345767
345779
330932
113059
SO/C 14885
3101278
W./C. 6608
SOTON/OQ 392086
19950
343275
343276
347466
W.E.P. 5734
C.A. 2315
364500
374910
PC 17754
PC 17759
231919
244367
349245
349215
35281
7540
3101276
349207
343120
312991
349249
371110
110465
2665
324669
4136
2627
STON/O 2. 3101294
370369
11668
PC 17558
347082
S.O.C. 14879
A4. 54510
237736
27267
35281
2651
370372
C 17369
2668
3470

In [108]:
preproc_data.groupby("LuckyTicket").agg('mean')

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
LuckyTicket,,,,,,,,,
False,445.076739,0.384892,2.298561,0.646283,29.479820,0.501199,0.363309,31.624419,1.544365
True,459.508772,0.368421,2.456140,0.666667,27.631579,0.842105,0.649123,40.687432,1.421053


In [109]:
(preproc_data["LuckyTicket"] == True).mean()

0.06397306397306397

In [ ]:
preproc_data.groupby("LuckyTicket").agg('mean')

In [110]:
preproc_data[preproc_data["Ticket"] == "LINE"]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,LuckyTicket
179,180,0,3,"Leonard, Mr. Lionel",1,36.0,0,0,LINE,0.0,2,False
271,272,1,3,"Tornquist, Mr. William Henry",1,25.0,0,0,LINE,0.0,2,False
302,303,0,3,"Johnson, Mr. William Cahoone Jr",1,19.0,0,0,LINE,0.0,2,False
597,598,0,3,"Johnson, Mr. Alfred",1,49.0,0,0,LINE,0.0,2,False


In [111]:
preproc_data.groupby("Pclass").agg('mean')

,PassengerId,Survived,Sex,Age,SibSp,Parch,Fare,Embarked,LuckyTicket
Pclass,,,,,,,,,
1,461.597222,0.629630,0.564815,36.812130,0.416667,0.356481,84.154687,1.203704,0.055556
2,445.956522,0.472826,0.586957,29.765380,0.402174,0.380435,20.662183,1.798913,0.038043
3,439.154786,0.242363,0.706721,25.932627,0.615071,0.393075,13.675550,1.584521,0.077393


## Идеи для дальнейших ноутбуков
1. Исследовать возможные способы заполнения отсутствующих вещественнозначных значений:
    1. Среднее значение по всем известным значениям
    2. Медианное значение по всем известным значениям
    3. Среднее/медианное значение по значениям тех записей, которые похожи на данную по некоторому другому признаку. Например., заполняем пропуск средним значением возраста людей того же пола.
    4. То же, что и в предыдущем пункте, но в качестве референсного признака попробовать отклик. Проверить, будет ли модель переобучаться сильнее.
2. Обучить модель на этих же данных (titanic), добавив в неё текстовые признаки (*Ticket* и *Name*). Проверить, улучшается ли качество.
    1. Одним из способов преобразования текстового признака *Ticket* будет разделение его на 2 признака: числовое значение номера билета и буквенная приписка в начале билета (она есть у некоторых записей). Итого получится 2 признака: вещественнозначный и категориальный соответственно (если у билета нет приписки, то это будет отдельный класс).
